In [1]:
from urllib.request import urlretrieve
import os
import numpy as np
import regex as regex
import codecs
from tqdm import tqdm
from collections import Counter
from sklearn import preprocessing
import datetime


Download text files of book in public domain.

In [2]:
book_files={
#     "Mickiewicz": [
#         "https://wolnelektury.pl/media/book/txt/pan-tadeusz.txt",
#         "https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt",
#         "https://wolnelektury.pl/media/book/txt/dziady-dziadow-czesci-iii-ustep-do-przyjaciol-moskali.txt",
#         "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-pani-twardowska.txt",
#         "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-powrot-taty.txt",
#         "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switez.txt",
#         "https://wolnelektury.pl/media/book/txt/dziady-dziady-poema-dziady-czesc-iv.txt",
#     ],
    "Sienkiewicz": [
        "https://wolnelektury.pl/media/book/txt/quo-vadis.txt",
        "https://wolnelektury.pl/media/book/txt/sienkiewicz-we-mgle.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/potop-tom-trzeci.txt",
    ],
    "Orzeszkowa": [
        "https://wolnelektury.pl/media/book/txt/orzeszkowa-kto-winien.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-trzeci.txt",
        "https://wolnelektury.pl/media/book/txt/gloria-victis-dziwna-historia.txt",
        "https://wolnelektury.pl/media/book/txt/z-pozogi.txt",
        "https://wolnelektury.pl/media/book/txt/pani-dudkowa.txt",
        "https://wolnelektury.pl/media/book/txt/dymy.txt",
        "https://wolnelektury.pl/media/book/txt/syn-stolarza.txt",
        "https://wolnelektury.pl/media/book/txt/dobra-pani.txt",
        "https://wolnelektury.pl/media/book/txt/cnotliwi.txt",
        "https://wolnelektury.pl/media/book/txt/kilka-slow-o-kobietach.txt",
        "https://wolnelektury.pl/media/book/txt/patryotyzm-i-kosmopolityzm.txt",
        "https://wolnelektury.pl/media/book/txt/julianka.txt",
    ],
    "Prus": [
        "https://wolnelektury.pl/media/book/txt/lalka-tom-drugi.txt",
        "https://wolnelektury.pl/media/book/txt/lalka-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/antek.txt",
        "https://wolnelektury.pl/media/book/txt/katarynka.txt",
        "https://wolnelektury.pl/media/book/txt/prus-anielka.txt",
        "https://wolnelektury.pl/media/book/txt/prus-placowka.txt",
        
    ],
    "Reymont": [
        "https://wolnelektury.pl/media/book/txt/ziemia-obiecana-tom-pierwszy.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-pierwsza-jesien.txt",
        "https://wolnelektury.pl/media/book/txt/reymont-chlopi-zima.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-trzecia-wiosna.txt",
        "https://wolnelektury.pl/media/book/txt/chlopi-czesc-czwarta-lato.txt",
    ]
}

In [3]:
for author in book_files:
    for url in book_files[author]:
        file_path = os.path.join("data",os.path.basename(url))
        if not os.path.exists(file_path):
            print("downloading {}".format(url))
            urlretrieve(url, file_path)
print ("DONE")

DONE


Preprocess files. Clean nonlatin characters, remove excessive spaces and newlines. Remove wolnelektury footer.


In [4]:
repl_dict={    
#     '.': '||period||',
    ',': ' ||comma||',
    '"': ' ||quotation_mark||',
    ';': ' ||semicolon||',
    '!': ' ||exclamation_mark||',
    '?': ' ||question_mark||',
    '(': ' ||left_parenthesis||',
    ')': ' ||right_parenthesis||',
    '--': ' ||dash||',
    '?': ' ||question_mark||',
#     '\n': '||return||'
}


def preprocess(text):
    """
    Perform a simple multiple replace.
    repl_dict has to be a dictionary i replace patterns.
    The key cannot contain patterns.
    """
    # build regexp
    reg_exp = regex.compile("|".join(map(regex.escape, repl_dict.keys())))

    # replace :)
    return reg_exp.sub(lambda match: repl_dict[match.group(0)], text)

In [5]:
# output corspus file with one sentence per line
def preprocess_file(file_path=None, file_url=None, overwrite=False):
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
        
    pp_file=file_path+".pp"
    if not os.path.exists(pp_file) or overwrite:
        text = open(file_path,'rb').read().lower().decode("utf-8").lower()

        text = regex.sub(u"[^ \n\p{Latin}\-'.?!]", " ",text)
        text = regex.sub(u"[ \n]+", " ", text) # Squeeze spaces and newlines
        text = regex.sub(r"----- ta lektura.*","", text) # remove footer
        text = preprocess(text)
        with codecs.open(pp_file, 'w', 'utf-8') as fout:
            for l in text.split('.'):
                l=regex.sub(r"^ ","",l) # remove leading spaces
                fout.write(l+" ||period||\n")
    return

for author in book_files:
    for url in book_files[author]:
        preprocess_file(file_url=url)
print ("DONE")

DONE


Count number of lines in each corpus.

In [6]:
def get_line_count(file_path=None, file_url=None):
    """
    Calculate numer of lines in the file
    """
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
    def blocks(files, size=65536):
        while True:
            b = files.read(size)
            if not b: break
            yield b

    with codecs.open(file_path, 'r', 'utf-8') as f:
        return sum(bl.count("\n") for bl in blocks(f))

linecounts={}

for author in book_files:
    linecounts[author]=0
    for url in book_files[author]:
        linecounts[author]+=get_line_count(file_url=url+".pp")

print(linecounts)
print(min(linecounts))
linecounts[min(linecounts)]

{'Sienkiewicz': 27632, 'Orzeszkowa': 14752, 'Prus': 21756, 'Reymont': 15786}
Orzeszkowa


14752

Get some stats.

In [7]:
def read_file_as_sentences(file_path=None, file_url=None, max_lines=2822, doc=[]):
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
    with open(file_path,"r", encoding="utf-8") as file:
        for l in file:
            doc.append(l[:-1])
    return doc

docs={}
for author in book_files:
    docs[author]=[]
    for url in book_files[author]:
        read_file_as_sentences(file_url=url+".pp", doc=docs[author])

sentence_lengths={a: np.array([len(l.split()) for l in docs[a]]) for a in docs}
print ("Mean words per sentence:")
print ({a: np.mean(sentence_lengths[a]) for a in sentence_lengths})
print ("Median words per sentence:")
print ({a: np.median(sentence_lengths[a]) for a in sentence_lengths})
print ("Max words per sentence:")
print ({a: np.max(sentence_lengths[a]) for a in sentence_lengths})
print ("Std dev words per sentence:")
print ({a: np.std(sentence_lengths[a]) for a in sentence_lengths})
        
for author in docs:
    np.random.shuffle(docs[author])

Mean words per sentence:
{'Sienkiewicz': 21.492291546033584, 'Orzeszkowa': 30.689126898047721, 'Prus': 18.751103143960286, 'Reymont': 26.537438236412012}
Median words per sentence:
{'Sienkiewicz': 17.0, 'Orzeszkowa': 24.0, 'Prus': 15.0, 'Reymont': 19.0}
Max words per sentence:
{'Sienkiewicz': 337, 'Orzeszkowa': 304, 'Prus': 276, 'Reymont': 522}
Std dev words per sentence:
{'Sienkiewicz': 16.443830141683854, 'Orzeszkowa': 24.966062018436322, 'Prus': 14.441170295378315, 'Reymont': 27.157258404042977}


In [8]:
mean_linecount=int(np.mean([linecounts[a] for a in linecounts]))
mean_linecount

19981

Balance all docs to the same number of sentences.

In [9]:
for author in docs:
    del(docs[author][mean_linecount:])
for author in docs:
    print(len(docs[author]))

19981
14752
19981
15786


In [10]:
vocab_size=10000

word_counts = Counter()
for author in docs:
    for line in docs[author]:
        word_counts.update(line.split())

        
vocabulary = dict(word_counts.most_common()[:vocab_size])


# Start at 2, 0 is for padding 1 is for unknown words
int_to_vocab = {word_id: word for word_id, word in enumerate(vocabulary,2)}
vocab_to_int = {word: word_id for word_id, word in int_to_vocab.items()}


In [11]:
print(len(vocabulary))

10000


Now build a train, verification and testing datasets, shuffle them and construct one hot encodded labels


In [12]:
def text_to_ids(source_text, vocab_to_int=vocab_to_int):
    """
    Convert text to proper word ids
    :param source_text: String that contains all the source text.
    :param vocab_to_int: Dictionary to go from the source words to an id
    :return: A lists id_text
    """
    
    # 1 is for Unknown words
    id_text = [[vocab_to_int.get(w,1) for w in l.split()] for l in source_text]
    
    return id_text

In [13]:
test = 0.1 # From all data
valid = 0.2 # of what's left

X_train=[]
X_valid=[]
X_test=[]
Y_train=[]
Y_valid=[]
Y_test=[]

#for each author chop off the same ammount of data from each author
for author in docs:
    sentences = len(docs[author])
    
    test_samples = int(sentences*test)
    valid_samples = int((sentences-test_samples)*valid)
    train_samples = sentences - valid_samples - test_samples
    print (train_samples, valid_samples, test_samples, author)
    
    # data
    X_train+=docs[author][:train_samples]
    X_valid+=docs[author][train_samples:train_samples+valid_samples]
    X_test+=docs[author][train_samples+valid_samples:]
    
    # labels
    Y_train+=[author for _ in range(train_samples)]
    Y_valid+=[author for _ in range(valid_samples)]
    Y_test+=[author for _ in range(test_samples)]


# Create the encoder
lb = preprocessing.LabelBinarizer()
# Here the encoder finds the classes and assigns one-hot vectors 
lb.fit(list(docs.keys()))



14387 3596 1998 Sienkiewicz
10622 2655 1475 Orzeszkowa
14387 3596 1998 Prus
11367 2841 1578 Reymont


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [14]:
# Convert to numbers
y_train =lb.transform(Y_train)
y_valid =lb.transform(Y_valid)
y_test = lb.transform(Y_test)


x_train = text_to_ids(X_train)
x_valid = text_to_ids(X_valid)
x_test = text_to_ids(X_test)


# reshuffle
def shuffle_dataset(dataset, labels):
    (dataset) = (np.array(dataset))
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

x_train, y_train = shuffle_dataset(x_train, y_train)
x_valid, y_valid = shuffle_dataset(x_valid, y_valid)
x_test, y_test = shuffle_dataset(x_test, y_test)

In [15]:
import warnings
from distutils.version import LooseVersion
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [16]:
def get_inputs(output_dim=4):
    """
    Create TF Placeholders for input, targets, learning_rate and input_sequence_length.
    :return: Tuple (input_, targets, learning_rate, keep_prob, input_sequence_length)
    """

    input_ = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.float32, [None, output_dim])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    is_training = tf.Variable(True)
    input_sequence_length = tf.placeholder(tf.int32, [None], name="input_sequence_length")
    
    return (input_, targets, learning_rate, keep_prob, input_sequence_length, is_training)

In [17]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob, inputs, vocab_size, input_sequence_length):
    ''' Build LSTM cell.
    
        Arguments
        ---------

        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        inputs: input tensor
        vocab_size: input vocabulary size
        input_sequence_length: input sequence length tensor
        
        Reurns
        ------
        touple (rnn_output, rnn_state, initial_state)
    '''
    
    # embed inputs
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    x_embed = tf.nn.embedding_lookup(embedding, inputs)
    
    def build_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.contrib.layers.xavier_initializer())
        return cell
    
    # Construct a stacked tf.contrib.rnn.LSTMCell...
    stacked_cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size) for _ in range(num_layers)])
    # ...wrapped in a tf.contrib.rnn.DropoutWrapper
    cell = tf.contrib.rnn.DropoutWrapper(stacked_cell, output_keep_prob=keep_prob)
    
    # Pass cell and embedded input to tf.nn.dynamic_rnn()
    rnn_output, rnn_state = tf.nn.dynamic_rnn(cell, x_embed, sequence_length=input_sequence_length, dtype=tf.float32)
    
    # Initial state
    initial_state = tf.identity(stacked_cell.zero_state(batch_size, tf.float32), name="initial_state")
    
    return rnn_output, rnn_state, initial_state

In [18]:
def build_output(cell, keep_prob, is_training, hidden_dim=1024, output_dim=3):
    input_ = cell[:, -1]
    dense = tf.contrib.layers.fully_connected(inputs=input_, 
                                              num_outputs=hidden_dim, 
                                              activation_fn=tf.nn.relu,
                                              weights_initializer=tf.contrib.layers.xavier_initializer()
                                             )
    dense = tf.layers.batch_normalization(dense, training=is_training)
    logits = tf.contrib.layers.fully_connected(dense, num_outputs=output_dim, activation_fn=None, 
                                            weights_initializer=tf.contrib.layers.xavier_initializer())
    return logits

In [19]:
model_name="1.0"
batch_size=128
lstm_size=512
num_layers=3
embed_dim=100
keep_probability=0.8

seq_len = 31+2*27  # Sequence Mean length + 2*sigma
output_dim=len(book_files)

learning_rate=0.0006

In [20]:
def get_padded_int_batch(input_batch):
    max_len = max([len(word) for word in input_batch])
    int_batch =  [[0] * (max_len - len(l)) + [w for w in l] for l in input_batch]
    return int_batch


def get_batch(input_list=x_train, output_list=y_train, batch_size=batch_size):
    """
    Batch generator.
    Input: train_set - list of words
    Returns touple:
    (pad_input_batch, pad_input_lengths, output_batch)
    """
    for batch_i in range(0, len(input_list)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        input_batch = input_list[start_i:start_i + batch_size]
        output_batch= output_list[start_i:start_i + batch_size]

        # Pad
        pad_input_batch = np.array(get_padded_int_batch(input_batch))

        # Need the lengths for the _lengths parameters
        pad_input_lengths = []
        for line in pad_input_batch:
            pad_input_lengths.append(len(line))
        
        yield (pad_input_batch, pad_input_lengths, output_batch)


In [21]:

def train(epoch_count, batch_size=batch_size, lstm_size=lstm_size, num_layers=num_layers, 
          keep_probability=keep_probability, vocab_size=vocab_size, output_dim=output_dim, 
          lr=learning_rate, model_name=model_name, display_step=10):

    experiment=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    param_string = "batch={}-lstm_size={}-num_layers={}-embed_dim={}-keep_probability={}-vocab_size={}-seq_len={}-output_dim={}-lr={}".format(
        batch_size, lstm_size, num_layers, embed_dim, keep_probability, vocab_size, seq_len, 
        output_dim,lr)
    save_dir = "writer/{}/{}/{}".format(model_name,experiment,param_string)
    print (save_dir)

    # construct model
    (input_, targets, learning_rate, keep_prob, input_sequence_length, is_training) = get_inputs(output_dim)
    
    with tf.variable_scope('LSTM'):
        rnn_output, rnn_state, initial_state = build_lstm(lstm_size, num_layers, batch_size, keep_prob, 
                                                          input_, vocab_size, input_sequence_length)
    with tf.variable_scope('LOGITS'):
        logits = build_output(rnn_output, keep_prob, is_training, output_dim=output_dim)
            
    with tf.variable_scope('LOSS'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=logits))
    
    with tf.variable_scope('OPTIMIZER'):
        optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(loss)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)


    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(targets, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Save session
        saver = tf.train.Saver()
        # Use Tensorboard for debuging
        writer=tf.summary.FileWriter(save_dir)
        writer.add_graph(sess.graph)
        training_loss_summary=tf.summary.scalar("training_loss", loss)
        training_accuracy_summary=tf.summary.scalar("training_accuracy", accuracy)
        validation_loss_summary=tf.summary.scalar("validation_loss", loss)
        validation_accuracy_summary=tf.summary.scalar("validation_accuracy", accuracy)
                         
        step=0
        for epoch_i in range(1, epoch_count):
            for batch_i, (pad_input_batch, pad_input_lengths, out_vec) in enumerate(get_batch()):

                
                if (batch_i % display_step ==0):
                    _, _, l, a, tls, tas = sess.run([initial_state, train_op, loss, accuracy, 
                                               training_loss_summary, training_accuracy_summary], {
                        input_: pad_input_batch,
                        targets: out_vec,
                        learning_rate: lr, 
                        keep_prob: keep_probability,
                        is_training: True,
                        input_sequence_length: pad_input_lengths,
                    })

                    # write summary for tensorboard
                    writer.add_summary(tls, step)
                    writer.add_summary(tas, step)
                    
                    (pad_input_batch, pad_input_lengths, out_vec) = next(
                        get_batch(input_list=x_valid, output_list=y_valid))
                    _, vl, va, vls, vas = sess.run([initial_state, loss, accuracy, 
                                                  validation_loss_summary, validation_accuracy_summary],{
                        input_: pad_input_batch,
                        targets: out_vec,
                        keep_prob: 1.0,
                        is_training: False,
                        input_sequence_length: pad_input_lengths,
                    })
                    # write summary for tensorboard
                    writer.add_summary(vls, step)
                    writer.add_summary(vas, step)
                    
                    step+=1

                    print("Epoch: {:3} | Loss: {:2.4}\t validation loss: {:2.4} | Accuracy: {:2.4}\t validation accuracy: {:2.4}".
                          format(epoch_i, l, vl, a, va))
                else:
                    _, _, = sess.run([initial_state, train_op], {
                        input_: pad_input_batch,
                        targets: out_vec,
                        learning_rate: lr, 
                        keep_prob: keep_probability,
                        is_training: True,
                        input_sequence_length: pad_input_lengths,
                    })

                    
            lr=lr*0.8 # (0.8^10 ~=0.1)
            saver.save(sess, save_dir, global_step=epoch_i)
 

In [22]:
with tf.Graph().as_default():
    train(10)

writer/1.0/2017-10-22 22:30:52/batch=128-lstm_size=512-num_layers=3-embed_dim=100-keep_probability=0.8-vocab_size=10000-seq_len=85-output_dim=4-lr=0.0006
Epoch:   1 | Loss: 1.437	 validation loss: 1.38 | Accuracy: 0.2422	 validation accuracy: 0.2812
Epoch:   1 | Loss: 1.556	 validation loss: 1.385 | Accuracy: 0.2344	 validation accuracy: 0.2344
Epoch:   1 | Loss: 1.497	 validation loss: 1.388 | Accuracy: 0.3047	 validation accuracy: 0.2344
Epoch:   1 | Loss: 1.428	 validation loss: 1.386 | Accuracy: 0.3828	 validation accuracy: 0.2344
Epoch:   1 | Loss: 1.55	 validation loss: 1.38 | Accuracy: 0.2422	 validation accuracy: 0.25
Epoch:   1 | Loss: 1.348	 validation loss: 1.376 | Accuracy: 0.4375	 validation accuracy: 0.2969
Epoch:   1 | Loss: 1.4	 validation loss: 1.412 | Accuracy: 0.375	 validation accuracy: 0.2578
Epoch:   1 | Loss: 1.428	 validation loss: 1.393 | Accuracy: 0.3359	 validation accuracy: 0.3047
Epoch:   1 | Loss: 1.251	 validation loss: 1.379 | Accuracy: 0.4219	 validatio

Epoch:   3 | Loss: 0.7614	 validation loss: 1.116 | Accuracy: 0.6953	 validation accuracy: 0.5078
Epoch:   3 | Loss: 0.8113	 validation loss: 1.164 | Accuracy: 0.6562	 validation accuracy: 0.5078
Epoch:   3 | Loss: 0.6388	 validation loss: 1.106 | Accuracy: 0.7188	 validation accuracy: 0.5156
Epoch:   3 | Loss: 0.7646	 validation loss: 1.096 | Accuracy: 0.6875	 validation accuracy: 0.5
Epoch:   3 | Loss: 0.7236	 validation loss: 1.149 | Accuracy: 0.7031	 validation accuracy: 0.5234
Epoch:   3 | Loss: 0.5854	 validation loss: 1.046 | Accuracy: 0.8047	 validation accuracy: 0.5547
Epoch:   3 | Loss: 0.734	 validation loss: 1.059 | Accuracy: 0.6953	 validation accuracy: 0.5469
Epoch:   3 | Loss: 0.6907	 validation loss: 1.108 | Accuracy: 0.7188	 validation accuracy: 0.4922
Epoch:   3 | Loss: 0.6623	 validation loss: 1.077 | Accuracy: 0.7578	 validation accuracy: 0.5234
Epoch:   3 | Loss: 0.6489	 validation loss: 1.075 | Accuracy: 0.7578	 validation accuracy: 0.4844
Epoch:   3 | Loss: 0.752

Epoch:   5 | Loss: 0.5046	 validation loss: 1.039 | Accuracy: 0.7812	 validation accuracy: 0.5234
Epoch:   5 | Loss: 0.3886	 validation loss: 1.02 | Accuracy: 0.8516	 validation accuracy: 0.5781
Epoch:   5 | Loss: 0.493	 validation loss: 0.9761 | Accuracy: 0.7734	 validation accuracy: 0.6094
Epoch:   5 | Loss: 0.5312	 validation loss: 0.9906 | Accuracy: 0.7656	 validation accuracy: 0.5781
Epoch:   5 | Loss: 0.5475	 validation loss: 0.964 | Accuracy: 0.8047	 validation accuracy: 0.6094
Epoch:   5 | Loss: 0.5194	 validation loss: 0.9549 | Accuracy: 0.8125	 validation accuracy: 0.6094
Epoch:   5 | Loss: 0.5886	 validation loss: 0.929 | Accuracy: 0.8125	 validation accuracy: 0.6328
Epoch:   5 | Loss: 0.4815	 validation loss: 0.9356 | Accuracy: 0.8203	 validation accuracy: 0.5938
Epoch:   5 | Loss: 0.5507	 validation loss: 0.9375 | Accuracy: 0.8047	 validation accuracy: 0.5625
Epoch:   5 | Loss: 0.4368	 validation loss: 0.9395 | Accuracy: 0.8281	 validation accuracy: 0.6016
Epoch:   5 | Los

Epoch:   7 | Loss: 0.476	 validation loss: 0.8827 | Accuracy: 0.8281	 validation accuracy: 0.625
Epoch:   7 | Loss: 0.4381	 validation loss: 0.8729 | Accuracy: 0.8438	 validation accuracy: 0.6406
Epoch:   7 | Loss: 0.5151	 validation loss: 0.8559 | Accuracy: 0.8359	 validation accuracy: 0.6484
Epoch:   7 | Loss: 0.3857	 validation loss: 0.8557 | Accuracy: 0.8359	 validation accuracy: 0.6641
Epoch:   7 | Loss: 0.4732	 validation loss: 0.8597 | Accuracy: 0.8438	 validation accuracy: 0.6406
Epoch:   7 | Loss: 0.3399	 validation loss: 0.859 | Accuracy: 0.8672	 validation accuracy: 0.6328
Epoch:   7 | Loss: 0.4924	 validation loss: 0.8706 | Accuracy: 0.8203	 validation accuracy: 0.6328
Epoch:   7 | Loss: 0.3416	 validation loss: 0.8789 | Accuracy: 0.8594	 validation accuracy: 0.6328
Epoch:   7 | Loss: 0.2539	 validation loss: 0.852 | Accuracy: 0.8906	 validation accuracy: 0.6484
Epoch:   7 | Loss: 0.4213	 validation loss: 0.8641 | Accuracy: 0.8438	 validation accuracy: 0.6484
Epoch:   7 | L

Epoch:   9 | Loss: 0.305	 validation loss: 0.8257 | Accuracy: 0.8984	 validation accuracy: 0.6875
Epoch:   9 | Loss: 0.4144	 validation loss: 0.8296 | Accuracy: 0.875	 validation accuracy: 0.6719
Epoch:   9 | Loss: 0.3031	 validation loss: 0.8311 | Accuracy: 0.8828	 validation accuracy: 0.6328
Epoch:   9 | Loss: 0.3965	 validation loss: 0.8411 | Accuracy: 0.8203	 validation accuracy: 0.6719
Epoch:   9 | Loss: 0.3116	 validation loss: 0.8678 | Accuracy: 0.875	 validation accuracy: 0.6328
Epoch:   9 | Loss: 0.2289	 validation loss: 0.8563 | Accuracy: 0.9219	 validation accuracy: 0.625
Epoch:   9 | Loss: 0.3644	 validation loss: 0.8402 | Accuracy: 0.8594	 validation accuracy: 0.6562
Epoch:   9 | Loss: 0.3149	 validation loss: 0.8525 | Accuracy: 0.8672	 validation accuracy: 0.6484
Epoch:   9 | Loss: 0.3172	 validation loss: 0.849 | Accuracy: 0.8672	 validation accuracy: 0.6484
Epoch:   9 | Loss: 0.3036	 validation loss: 0.8505 | Accuracy: 0.8594	 validation accuracy: 0.6406
Epoch:   9 | Lo